In [1]:
import _threshold_utils as threshold_utils
import _speed_utils as speed_utils
import altair as alt

import pandas as pd
import geopandas as gpd
from segment_speed_utils.project_vars import analysis_date
from shared_utils import calitp_color_palette as cp

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/home/jovyan/data-analyses/rt_segment_speeds/_threshold_utils.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/lat

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
GCS_PATH = "gs://calitp-analytics-data/data-analyses/rt_segment_speeds/"

In [4]:
analysis_date

'2023-03-15'

In [5]:
# Flag
routes_many_stops_df, routes_many_stops_list = speed_utils.find_shapes_with_many_stops(analysis_date)

14517 duplicate rows will be dropped
0    4512
Name: stops_renumbered_correctly, dtype: int64


In [6]:
m1 =  speed_utils.merge_all_speeds(analysis_date)

14517 duplicate rows will be dropped
0    4512
Name: stops_renumbered_correctly, dtype: int64


In [7]:
m1.shape

(2779187, 12)

In [8]:
m1.shape_array_key.nunique()

4512

In [9]:
speeds_stop_segments = pd.read_parquet(f"{speed_utils.GCS_PATH}avg_speeds_stop_segments_{analysis_date}.parquet").drop(columns = ['geometry','geometry_arrowized'])

In [10]:
speeds_stop_segments_geo = gpd.read_parquet(f"{speed_utils.GCS_PATH}avg_speeds_stop_segments_{analysis_date}.parquet")

In [11]:
avg_speeds = pd.read_parquet(f"{speed_utils.GCS_PATH}speeds_stop_segments_{analysis_date}")

In [12]:
avg_speeds.sample()

,gtfs_dataset_key,_gtfs_dataset_name,shape_array_key,stop_sequence,trip_id,min_time,min_dist,max_time,max_dist,meters_elapsed,sec_elapsed,speed_mph
__null_dask_index__,,,,,,,,,,,,
2419167,7342e9ab4e268b54f9cce8cc25f11345,Foothill Vehicle Positions,23ccb2b031fcc0661dd1a6528a0044c0,1296,t2F3-b27C8-sl5,2023-03-15 08:15:20,56.61,2023-03-15 08:19:03,1874.10,1817.49,223.00,18.23


In [15]:
# speeds_stop_segments['minutes_elapsed'] = og.sec_elapsed/60

In [16]:
# speeds_stop_segments.minutes_elapsed.describe()

In [17]:
# speeds_stop_segments.head()

### Look through

In [18]:
m1.groupby(['shape_array_key']).agg({'trip_id':'nunique'}).sort_values(['trip_id'], ascending = False).head()

,trip_id
shape_array_key,
6388c0be232f0c745df85d66689a6db0,208
2c546e0dc8375fdb23c4b987ed47f968,166
9ed053064d883cca43d29dea9098def2,163
d7cc7b423029e8fb97c6695e45c0997d,163
d9a8b04ff20d6812ea6e22f655580680,159


In [19]:
one_route_geo = speeds_stop_segments_geo.loc[speeds_stop_segments_geo.shape_array_key == "6388c0be232f0c745df85d66689a6db0"].reset_index(drop = True)

In [20]:
one_route_geo = pd.merge(one_route_geo,avg_speeds, on = ['gtfs_dataset_key','shape_array_key', 'stop_sequence'], how = "inner")

In [21]:
type(one_route_geo)

geopandas.geodataframe.GeoDataFrame

In [22]:
one_route_geo = one_route_geo.drop(columns = ['min_time',
       'min_dist', 'max_time', 'max_dist',])

In [23]:
one_route_geo.columns

Index(['shape_array_key', 'stop_sequence', 'gtfs_dataset_key',
       'loop_or_inlining', 'geometry', 'geometry_arrowized', 'avg_speed_mph',
       'n_trips', 'p20_speed_mph', 'p80_speed_mph', 'district',
       'district_name', '_gtfs_dataset_name', 'trip_id', 'meters_elapsed',
       'sec_elapsed', 'speed_mph'],
      dtype='object')

In [24]:
one_route_geo = one_route_geo[one_route_geo.trip_id.isin(["57678514", "57678511"])]

In [25]:
one_route_geo.explore('trip_id')

In [26]:
one_route = m1.loc[m1.shape_array_key == "6388c0be232f0c745df85d66689a6db0"].reset_index(drop = True)

In [27]:
one_route[one_route.sorted_stop_seq == 6]

,shape_array_key,stop_sequence,gtfs_dataset_key,loop_or_inlining,avg_speed_mph,n_trips,p20_speed_mph,p80_speed_mph,_gtfs_dataset_name,trip_id,speed_mph,sorted_stop_seq
5,6388c0be232f0c745df85d66689a6db0,7,eded866f867cc3292c125d782c1698bb,1,1.04,2,0.82,1.27,LA Metro Rail Vehicle Positions,57678511,0.66,6


In [28]:
one_route

,shape_array_key,stop_sequence,gtfs_dataset_key,loop_or_inlining,avg_speed_mph,n_trips,p20_speed_mph,p80_speed_mph,_gtfs_dataset_name,trip_id,speed_mph,sorted_stop_seq
0,6388c0be232f0c745df85d66689a6db0,1,eded866f867cc3292c125d782c1698bb,1,0.30,2,0.26,0.33,LA Metro Rail Vehicle Positions,57678511,0.36,1
1,6388c0be232f0c745df85d66689a6db0,2,eded866f867cc3292c125d782c1698bb,1,0.30,2,0.26,0.33,LA Metro Rail Vehicle Positions,57678511,0.36,2
2,6388c0be232f0c745df85d66689a6db0,3,eded866f867cc3292c125d782c1698bb,1,5.76,103,4.17,7.28,LA Metro Rail Vehicle Positions,57678511,0.75,3
3,6388c0be232f0c745df85d66689a6db0,4,eded866f867cc3292c125d782c1698bb,1,11.54,105,9.43,13.72,LA Metro Rail Vehicle Positions,57678511,2.02,4
4,6388c0be232f0c745df85d66689a6db0,6,eded866f867cc3292c125d782c1698bb,1,1.05,2,0.83,1.28,LA Metro Rail Vehicle Positions,57678511,0.68,5
5,6388c0be232f0c745df85d66689a6db0,7,eded866f867cc3292c125d782c1698bb,1,1.04,2,0.82,1.27,LA Metro Rail Vehicle Positions,57678511,0.66,6
6,6388c0be232f0c745df85d66689a6db0,8,eded866f867cc3292c125d782c1698bb,1,1.04,2,0.81,1.27,LA Metro Rail Vehicle Positions,57678511,0.65,7
7,6388c0be232f0c745df85d66689a6db0,3,eded866f867cc3292c125d782c1698bb,1,5.76,103,4.17,7.28,LA Metro Rail Vehicle Positions,57678512,3.25,1
8,6388c0be232f0c745df85d66689a6db0,4,eded866f867cc3292c125d782c1698bb,1,11.54,105,9.43,13.72,LA Metro Rail Vehicle Positions,57678512,7.20,2
9,6388c0be232f0c745df85d66689a6db0,3,eded866f867cc3292c125d782c1698bb,1,5.76,103,4.17,7.28,LA Metro Rail Vehicle Positions,57678514,0.31,1


In [29]:
one_route[one_route.stop_sequence == 7]

,shape_array_key,stop_sequence,gtfs_dataset_key,loop_or_inlining,avg_speed_mph,n_trips,p20_speed_mph,p80_speed_mph,_gtfs_dataset_name,trip_id,speed_mph,sorted_stop_seq
5,6388c0be232f0c745df85d66689a6db0,7,eded866f867cc3292c125d782c1698bb,1,1.04,2,0.82,1.27,LA Metro Rail Vehicle Positions,57678511,0.66,6
12,6388c0be232f0c745df85d66689a6db0,7,eded866f867cc3292c125d782c1698bb,1,1.04,2,0.82,1.27,LA Metro Rail Vehicle Positions,57678514,1.42,4


In [30]:
one_route[one_route.trip_id.isin(["57678514", "57678511"])].sort_values(by = ['trip_id','stop_sequence','sorted_stop_seq'])[['trip_id','stop_sequence','sorted_stop_seq']]

,trip_id,stop_sequence,sorted_stop_seq
0,57678511,1,1
1,57678511,2,2
2,57678511,3,3
3,57678511,4,4
4,57678511,6,5
5,57678511,7,6
6,57678511,8,7
9,57678514,3,1
10,57678514,4,2
11,57678514,6,3


In [31]:
one_route.std()

/tmp/ipykernel_877/2789643931.py:1: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.


stop_sequence       7.19
loop_or_inlining    0.00
avg_speed_mph       8.25
n_trips            27.18
p20_speed_mph       6.18
p80_speed_mph       9.74
speed_mph          10.15
sorted_stop_seq     0.80
dtype: float64

In [32]:
one_route.groupby(['gtfs_dataset_key','shape_array_key','stop_sequence'])['speed_mph'].std()

gtfs_dataset_key                  shape_array_key                   stop_sequence
eded866f867cc3292c125d782c1698bb  6388c0be232f0c745df85d66689a6db0  1                0.08
                                                                    2                0.08
                                                                    3                2.83
                                                                    4                4.21
                                                                    5                7.88
                                                                    6                0.52
                                                                    7                0.54
                                                                    8                0.54
                                                                    17               3.92
                                                                    18              10.98
                  

In [33]:
one_route.stop_sequence.value_counts()

4     105
3     103
17    102
19     96
18     33
5      27
1       2
2       2
6       2
7       2
8       2
Name: stop_sequence, dtype: int64

In [34]:
one_route.sorted_stop_seq.value_counts()

1    208
2    200
3     61
4      3
5      2
6      1
7      1
Name: sorted_stop_seq, dtype: int64

In [35]:
 one_route.trip_id.value_counts().describe()

count   208.00
mean      2.29
std       0.65
min       1.00
25%       2.00
50%       2.00
75%       3.00
max       7.00
Name: trip_id, dtype: float64

In [36]:
one_route.groupby(['gtfs_dataset_key','shape_array_key','sorted_stop_seq'])['speed_mph'].std()

gtfs_dataset_key                  shape_array_key                   sorted_stop_seq
eded866f867cc3292c125d782c1698bb  6388c0be232f0c745df85d66689a6db0  1                  5.06
                                                                    2                 11.76
                                                                    3                  9.31
                                                                    4                  0.40
                                                                    5                  0.52
                                                                    6                   NaN
                                                                    7                   NaN
Name: speed_mph, dtype: float64

In [37]:
one_route.groupby(['gtfs_dataset_key','shape_array_key','stop_sequence'])['speed_mph'].std()

gtfs_dataset_key                  shape_array_key                   stop_sequence
eded866f867cc3292c125d782c1698bb  6388c0be232f0c745df85d66689a6db0  1                0.08
                                                                    2                0.08
                                                                    3                2.83
                                                                    4                4.21
                                                                    5                7.88
                                                                    6                0.52
                                                                    7                0.54
                                                                    8                0.54
                                                                    17               3.92
                                                                    18              10.98
                  

### Charts  
Test with a few routes first
* Create new col that rounds up speed for plotting purposes only.

#### Manipulate DF for charts

In [38]:
test1 = m1.melt(id_vars=[ '_gtfs_dataset_name','shape_array_key','trip_id', 'sorted_stop_seq','gtfs_dataset_key','loop_or_inlining',
                       'n_trips'], value_vars=[ 'avg_speed_mph','speed_mph','p20_speed_mph', 'p80_speed_mph'])

In [39]:
test1.shape

(11116748, 9)

In [40]:
# test1[test1.shape_array_key == "29d2bbdbeaec1d6888800f85bebf6e33"]

In [41]:
# Only need average speed/p20 speed/p80 to show up once for each stop sequence-operator-shape array
test2 = test1.drop_duplicates(subset = [ '_gtfs_dataset_name','shape_array_key','sorted_stop_seq','gtfs_dataset_key','variable','value']).reset_index(drop = True)

In [42]:
# test2.to_csv("./speeds.csv")

In [43]:
test2.shape

(4189930, 9)

In [44]:
other = ['cf688717cf0cd8dac0e6d1f12f9c7333',
       '6f39f818c9a0c5496cd1c8bd1aa11e67',
       '3de4482ec32ba0f2edb451d3528b5a5e']

In [45]:
# Take out routes that have over 85 stops
# subset = test2[~test2.shape_array_key.isin(routes_many_stops_list)].reset_index(drop = True)

In [46]:
subset = test2[test2.shape_array_key.isin(["29d2bbdbeaec1d6888800f85bebf6e33",'754c5b012195800c38dc58e72e4f482e',
      'e3c5ed2c6fa6cd5c5cd57d46aeb3cd8e', ])]

In [47]:
subset = threshold_utils.pre_clean(subset)

In [48]:
subset.columns

Index(['Gtfs Dataset Name', 'Shape Array Key', 'Trip Id', 'Sorted Stop Seq',
       'Gtfs Dataset Key', 'Loop Or Inlining', 'N Trips', 'Variable', 'Value'],
      dtype='object')

In [49]:
subset.sample()

,Gtfs Dataset Name,Shape Array Key,Trip Id,Sorted Stop Seq,Gtfs Dataset Key,Loop Or Inlining,N Trips,Variable,Value
996086,Anaheim Resort VehiclePositions,29d2bbdbeaec1d6888800f85bebf6e33,aa6a03fe-f119-44a5-8976-2ad77ceeb89f:1,4,1051e067d18a76b0a79789f051d5c22b,1,8,speed_mph,8.90


In [50]:
subset['Route'] = subset['Gtfs Dataset Name'] +' '  + subset['Shape Array Key']

In [51]:
subset = subset.rename(columns = {'Value':'Speed'})

In [52]:
subset['Speed_Int'] = subset.Speed.fillna(0).astype(int)

In [53]:
subset['Route Type'] = "Loop or Inlining: " + subset["Loop Or Inlining"].astype(str)

In [54]:
# subset['Rounded Speed'].unique()

In [55]:
def speed(row):
    # If partner is none, return Unknown.
    if row.Speed_Int == 0:
        return 0
    elif 0 < row.Speed_Int < 6:
        return 5
    elif 5 < row.Speed_Int < 11:
        return 10
    elif 10 < row.Speed_Int < 16:
        return 15
    elif 15 < row.Speed_Int < 21:
        return 20
    elif 20 < row.Speed_Int < 26:
        return 25
    elif 25 < row.Speed_Int < 31:
        return 30
    else:
        return 35

In [56]:
# Apply the function
subset["Rounded Speed"] = subset.apply(speed, axis=1)

In [57]:
# subset[['Rounded Speed', 'Speed', 'Speed_Int']]

In [58]:
subset.Variable = subset.Variable.str.title().str.replace("_"," ")

In [59]:
# One df for the actual speeds
subset_speedmph = subset[subset.Variable == 'Speed Mph'].reset_index(drop = True)

In [60]:
# One df for the percentiles
subset_other= subset[subset.Variable != 'Speed Mph'].reset_index(drop = True)

In [61]:
def alt_dropdown(df, col_for_dropdown:str, dropdown_menu_title:str):
    # Create dropdown menu
    # Exclude "none" operators which are only scheduled data
    df = df.loc[df[col_for_dropdown] != "None"][[col_for_dropdown]]
    dropdown_list = df[col_for_dropdown].unique().tolist()
    
    # Show only first operator by default
    initialize_first_op = sorted(dropdown_list)[0]
    input_dropdown = alt.binding_select(options=sorted(dropdown_list), name=dropdown_menu_title)
    
    selection = alt.selection_single(name= dropdown_menu_title,fields=[col_for_dropdown],
    bind=input_dropdown, init={col_for_dropdown: initialize_first_op})
                 
    return selection

In [62]:
selection_test = alt_dropdown(subset, "Route", "Operator/Shape Array")

In [63]:
# https://github.com/altair-viz/altair/issues/1168
title = alt.Chart(subset).mark_text(dy=-40, size=15, fontWeight='normal').encode(
    text='Route Type:N',
)

In [64]:
title = title.add_selection(selection_test).transform_filter(selection_test)

#### Scatterplot

#### Jitter

In [65]:
def create_jitter_plot(df):
    
    #title_op = df['Gtfs Dataset Name'].iloc[0].replace('VehiclePositions','').strip()
    #inline = df['Loop Or Inlining'].iloc[0]
    chart1 = (
        alt.Chart(df, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("Rounded Speed:Q",
                    scale=alt.Scale(domain=[0,50]),
                    title = "Speed (MPH)",
                    axis=alt.Axis(labelAngle=360, grid=False,)),
            color=alt.Color(
                "Variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=df.columns.tolist(),
            column=alt.Column(
                "Sorted Stop Seq:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="top",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title="Speeds by Operator-Shape Array")
    )
    
    chart1 = threshold_utils.chart_size(chart1, 75, 200)
    
    return chart1

In [66]:
chart1 = create_jitter_plot(subset_speedmph).add_selection(selection_test).transform_filter(selection_test)

In [67]:
chart2 = (
        alt.Chart(subset_other, width=0.5)
        .mark_circle(size=200)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("Rounded Speed:Q", 
                    title = "Speed (MPH)",
                    scale=alt.Scale(domain=[0, 50]),
                   axis = alt.Axis(grid = False)),
            color=alt.Color(
                "Variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=subset_other.columns.tolist(),
            column=alt.Column(
                "Sorted Stop Seq:N",
                header=alt.Header(
                    labelAngle=360,
                    title = None,
                    titleOrient="top",
                    labelOrient="top",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
    )
    

In [68]:
chart2 = threshold_utils.chart_size(chart2, 75, 200)

In [69]:
chart2 = chart2.add_selection(selection_test).transform_filter(selection_test)

In [70]:
title = threshold_utils.chart_size(title, 20, 20)

In [71]:
alt.data_transformers.enable('default', max_rows=None)

DataTransformerRegistry.enable('default')

In [72]:
title & (chart1.interactive() & chart2.interactive())

alt.VConcatChart(...)

### Draft

In [73]:
"""
p25 = troubleshoot.total_stops.quantile(0.25).astype(float)
p50 =  troubleshoot.total_stops.quantile(0.50).astype(float)
p75 =  troubleshoot.total_stops.quantile(0.75).astype(float)
p95 =  troubleshoot.total_stops.quantile(0.95).astype(float)
p99 =  troubleshoot.total_stops.quantile(0.99).astype(float)
"""

'\np25 = troubleshoot.total_stops.quantile(0.25).astype(float)\np50 =  troubleshoot.total_stops.quantile(0.50).astype(float)\np75 =  troubleshoot.total_stops.quantile(0.75).astype(float)\np95 =  troubleshoot.total_stops.quantile(0.95).astype(float)\np99 =  troubleshoot.total_stops.quantile(0.99).astype(float)\n'

In [74]:
def stop_categories1(row):
        if ((row.total_stops > 0) and (row.total_stops <= p25)):
            return "25th  <= 17 stops"
        elif ((row.total_stops > p25) and (row.total_stops <= p75)):
            return "50th <= 30 stops"
        elif ((row.total_stops > p75) and (row.total_stops <= p95)):
               return "75th <= 50 stops"
        elif ((row.total_stops > p95) and (row.total_stops <= p99)):
               return "95th <= 85 stops"
        elif row.total_stops >= p95:
               return "99th >= 203 stops"
        else:
            return "other"

In [75]:
def create_jitter_plot(df):
    
    title_op = df['Gtfs Dataset Name'].iloc[0].replace('VehiclePositions','').strip()
    inline = df['Loop Or Inlining'].iloc[0]
    
    chart1 = (
        alt.Chart(df, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("Rounded Speed:Q", axis=alt.Axis(labelAngle=360)),
            color=alt.Color(
                "Variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=df.columns.tolist(),
            column=alt.Column(
                "Stop Sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title=f"{title_op} - Route Type {inline}")
    )
    
    chart1 = threshold_utils.chart_size(chart1, 40, 250)
    
    return chart1

In [76]:
chart2 = (
        alt.Chart(anaheim_test, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .configure_facet(spacing=0)
        .configure_view(stroke=None)
        .properties(title="Trip Duration by RT Category")
    )

NameError: name 'anaheim_test' is not defined

In [ ]:
chart2 = threshold_utils.chart_size(chart2,80,300)

In [ ]:
chart2

In [ ]:
chart1 = (
        alt.Chart(anaheim_test_speedmph, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "stop_sequence:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .configure_facet(spacing=0)
        .configure_view(stroke=None)
        .properties(title=f"Loop/Inling:{anaheim_test_speedmph.loop_or_inlining.iloc[0]}")
    )

In [ ]:
chart1 = threshold_utils.chart_size(chart1,80,300)

In [ ]:
chart1

In [ ]:
def create_dot_plot2(df, col_for_dots: str, 
                    x_axis_col:str, y_axis_col:str,
                   tooltip_cols:list, chart_title:str):
  
    chart = (alt.Chart(df).mark_circle(opacity=1, size = 100).transform_window(
    id='rank()',
    groupby=[col_for_dots]).encode(
    alt.X(f'{x_axis_col}:O', sort='descending', 
          axis=alt.Axis(ticks=False, grid=True)),
    alt.Y(f'{y_axis_col}:N'), 
    color=alt.Color(f"{col_for_dots}:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS), legend=None),
    tooltip = tooltip_cols)
             .properties(title = chart_title))
    
    return chart

In [ ]:
chart3 = create_dot_plot1(anaheim_test_other, 'variable', 'stop_sequence', 'rounded_speed', anaheim_test_other.columns.tolist(),  'Percentile/Average')

In [ ]:
chart3 = threshold_utils.chart_size(chart3,650,300)

In [ ]:
chart4 = create_dot_plot2(anaheim_test_speedmph, 'variable', 'stop_sequence', 'rounded_speed', anaheim_test_speedmph.columns.tolist(), 'Speed per Trip')

In [ ]:
chart4 = threshold_utils.chart_size(chart4,650,300)

In [ ]:
chart4

In [ ]:
chart3 + chart4

In [ ]:
chart7 = (
        alt.Chart(anaheim_test_other, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=-90,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .configure_facet(spacing=0)
        .configure_view(stroke=None)
        .properties(title="Trip Duration by RT Category")
    )

In [ ]:
chart7 = threshold_utils.chart_size(chart7,80,300)

In [ ]:
chart8 = (
        alt.Chart(anaheim_test_other, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=-90,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title="Trip Duration by RT Category")
    )

In [ ]:
chart9 = (
        alt.Chart(anaheim_test_speedmph, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title=f"Loop/Inling:{anaheim_test_speedmph.loop_or_inlining.iloc[0]}")
    )

In [ ]:
chart8

In [ ]:
chart9 | chart8

In [ ]:
# pip install altair==5.0.0rc3
chart5 = alt.Chart(anaheim_test_speedmph, title='Normally distributed jitter').mark_circle(size=50).encode(
    y="rounded_speed:Q",
    x="stop_sequence:N",
    yOffset="jitter:Q",
    color=alt.Color('stop_sequence:Q').legend(None)
).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
)


In [ ]:
chart5 = threshold_utils.chart_size(chart5,650,300)

In [ ]:
chart5

#### Look at one trip

In [ ]:
#foothill_og = speed_stops2[speed_stops2.trip_id == "t604-b2791-sl5"]

In [ ]:
#len(foothill_og)

In [ ]:
#foothill_og.stop_sequence.nunique()

In [ ]:
#foothill_og.stop_sequence.describe()

In [ ]:
#foothill_og.sort_values('stop_sequence').head()

In [ ]:
#foothill_renumbered_stop_seq = m2[m2.trip_id == "t604-b2791-sl5"] 

In [ ]:
#foothill_renumbered_stop_seq['Test Stop Sequence'].describe()

In [ ]:
#foothill_renumbered_stop_seq.sort_values('stop_sequence').head()

In [ ]:
# len(troubleshoot)

In [ ]:
# Number of test stops should match stop sequence...
# troubleshoot['sequences_are_equal'] = troubleshoot['Test Stop Sequence'] - troubleshoot['stop_sequence']

In [ ]:
# troubleshoot['sequences_are_equal'].value_counts()

In [ ]:
# Look at this trip id in the original df
#og_trip = speed_stops2[speed_stops2.trip_id == "t640-b15FF1-sl5"]

In [ ]:
# Look at this trip id in the manipulated df
#new_trip = m2[m2.trip_id == "t640-b15FF1-sl5"]

In [ ]:
#og_trip.shape, og_trip.stop_sequence.nunique()

In [ ]:
#new_trip.shape, new_trip.stop_sequence.nunique()